# LAB 1
## Roxane GOFFINET

Comparison and implementation of different algorithm to explore and cover sets.

A part of the code is comimg from/inspired from M. Squillero public works.



In [1]:
import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue
import numpy as np
import pandas as pd


In [2]:
State = namedtuple('State', ['taken', 'not_taken'])


def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))

PROBLEM_SIZE = 250
NUM_SETS = 5000
SETS = tuple(np.array([random.random() < 0.3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"


# Definition of a DataFrame to save the results
res = pd.DataFrame(index = ['steps', 'tiles', 'special_steps', 'special_tiles'], columns=['Depth_First', 'Breadth_First', 'Greedy_Best_First', 'Astar_h1','Astar_h2','Astar_h3'])


## Depth First

Very fast : generally less than 1 sec

In [3]:
frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.pop()

while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.append(new_state)
    current_state = frontier.pop()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

res['Depth_First'].loc['steps']= counter
res['Depth_First'].loc['tiles']= len(current_state.taken)

Solved in 13 steps (13 tiles)


## Breadth First


It is rather long (around 5 minutes on my personal laptop)

In [ ]:
frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.popleft()

while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.append(new_state)
    current_state = frontier.popleft()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

#res['Breadth_First'].loc['steps']= counter
#res['Breadth_First'].loc['tiles']= len(current_state.taken)

: 

## Greedy Best First

In [4]:
def f(state):
    missing_size = PROBLEM_SIZE - sum(covered(state))
    return missing_size

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

res['Greedy_Best_First'].loc['steps']= counter
res['Greedy_Best_First'].loc['tiles']= len(current_state.taken)

Solved in 4 steps (4 tiles)


## A*

In [5]:
# Definition of 3 different heurisics to test

def h1(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


In [6]:
# Heuristic 1


def f(state):
    return len(state.taken) + h1(state)

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

res['Astar_h1'].loc['steps']= counter
res['Astar_h1'].loc['tiles']= len(current_state.taken)


Solved in 3,510 steps (4 tiles)


In [7]:
# Heuristic 2


def f(state):
    return len(state.taken) + h2(state)

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

res['Astar_h2'].loc['steps']= counter
res['Astar_h2'].loc['tiles']= len(current_state.taken)

Solved in 385 steps (4 tiles)


In [8]:
# Heuristic 3


def f(state):
    return len(state.taken) + h3(state)

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

res['Astar_h3'].loc['steps']= counter
res['Astar_h3'].loc['tiles']= len(current_state.taken)

Solved in 383 steps (4 tiles)


### Comparison of the algorithm

In [9]:
res

,Depth_First,Breadth_First,Greedy_Best_First,Astar_h1,Astar_h2,Astar_h3
steps,13,NaN,4,3510,385,383
tiles,13,NaN,4,4,4,4
special_steps,NaN,NaN,NaN,NaN,NaN,NaN
special_tiles,NaN,NaN,NaN,NaN,NaN,NaN


### Special_sets



In [10]:
# Definition of a special set to test the heuristics

SETS = [tuple([False] * PROBLEM_SIZE) for _ in range(NUM_SETS)]
random_set_index = random.randint(0, NUM_SETS - 1)
SETS[random_set_index] = tuple([True] * PROBLEM_SIZE)
print(f"The set with all True values is at index {random_set_index}")
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

The set with all True values is at index 6


#### Depth First

In [11]:
frontier = deque()
state = State(set(), set(range(NUM_SETS)))
frontier.append(state)

counter = 0
current_state = frontier.pop()

while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.append(new_state)
    current_state = frontier.pop()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

res['Depth_First'].loc['special_steps']= counter
res['Depth_First'].loc['special_tiles']= len(current_state.taken)

Solved in 34 steps (34 tiles)


##### Geedy Best First

In [12]:
def f(state):
    missing_size = PROBLEM_SIZE - sum(covered(state))
    return missing_size

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

res['Greedy_Best_First'].loc['special_steps']= counter
res['Greedy_Best_First'].loc['special_tiles']= len(current_state.taken)

Solved in 1 steps (1 tiles)


##### A* heuristic 1

In [13]:
def f(state):
    return len(state.taken) + h1(state)

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

res['Astar_h1'].loc['special_steps']= counter
res['Astar_h1'].loc['special_tiles']= len(current_state.taken)

Solved in 1 steps (1 tiles)


##### A* Heuristic 2

In [14]:

def f(state):
    return len(state.taken) + h2(state)

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

res['Astar_h2'].loc['special_steps']= counter
res['Astar_h2'].loc['special_tiles']= len(current_state.taken)

Solved in 1 steps (1 tiles)


##### A* heuristic 3

In [15]:
# Heuristic 3


def f(state):
    return len(state.taken) + h3(state)

frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

res['Astar_h3'].loc['special_steps']= counter
res['Astar_h3'].loc['special_tiles']= len(current_state.taken)

Solved in 1 steps (1 tiles)


### Comparison of the algorithms

In [16]:
res

,Depth_First,Breadth_First,Greedy_Best_First,Astar_h1,Astar_h2,Astar_h3
steps,13,NaN,4,3510,385,383
tiles,13,NaN,4,4,4,4
special_steps,34,NaN,1,1,1,1
special_tiles,34,NaN,1,1,1,1


As we can see the Depth first algorithm is very fast but it uses a lot of steps and tiles. Greedy Best first is also fast and  uses not too much steps and tiles. The Astar always give the minimum number of tiles used but genrally it takes more steps to get there. It is also the best algorithm to handle special sets (Depth First is very bad at that)